In [ ]:
pip install flask numpy joblib scikit-learn


In [25]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

# Load the dataset
df = pd.read_csv('/content/Sleep_health_and_lifestyle_dataset_transformed.csv')

# Preprocessing steps
df['BMI_Underweight'] = df['BMI_Underweight'].astype(int)
df['BMI_Normal'] = df['BMI_Normal'].astype(int)
df['BMI_Obese'] = df['BMI_Obese'].astype(int)
df['BMI_Overweight'] = df['BMI_Overweight'].astype(int)

df.rename(columns={
    'Age': 'age',
    'Gender': 'gender',
    'Stress Level': 'stress_level',
    'Sleep Duration': 'sleep_duration',
    'BMI_Underweight': 'bmi_underweight',
    'BMI_Normal': 'bmi_normal',
    'BMI_Obese': 'bmi_obese',
    'BMI_Overweight': 'bmi_overweight',
    'Quality of Sleep': 'quality_score'
}, inplace=True)

df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

X = df[['age', 'gender', 'stress_level', 'sleep_duration', 'bmi_underweight', 'bmi_normal', 'bmi_overweight', 'bmi_obese']]
y = df['quality_score']

# Normalize features using MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler for later use
joblib.dump(scaler, 'scaler.pkl')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define and compile the neural network model
nn_model = Sequential()
nn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(1, activation='linear'))
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the neural network model
nn_model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Evaluate the neural network model
nn_mse = nn_model.evaluate(X_test, y_test)
print(f"Neural Network MSE: {nn_mse}")

# Train and evaluate a linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
lr_mse = mean_squared_error(y_test, y_pred_lr)
print(f"Linear Regression MSE: {lr_mse}")

# Perform cross-validation
cv_scores = cross_val_score(lr_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validation MSE: {-cv_scores.mean()}")

# Predict quality score with the new data
new_data = {
    "age": 28,
    "gender": 1,
    "stress_level": 8,
    "sleep_duration": 5.9,
    "bmi_underweight": 0,
    "bmi_normal": 0,
    "bmi_overweight": 0,
    "bmi_obese": 1
}

new_data_df = pd.DataFrame([new_data])
new_data_scaled = scaler.transform(new_data_df)
predicted_quality_score = nn_model.predict(new_data_scaled)
print(f"Predicted Quality Score: {predicted_quality_score[0][0]}")


# Save the model to an HDF5 file
model_path = '/content/neural_network_model.h5'
nn_model.save(model_path)
print(f"Model saved to {model_path}")


Epoch 1/100
30/30 [==============================] - 2s 14ms/step - loss: 43.0053 - val_loss: 19.2155
Epoch 2/100
30/30 [==============================] - 0s 6ms/step - loss: 6.7367 - val_loss: 2.0682
Epoch 3/100
30/30 [==============================] - 0s 7ms/step - loss: 1.2958 - val_loss: 0.5916
Epoch 4/100
30/30 [==============================] - 0s 7ms/step - loss: 0.7149 - val_loss: 0.3884
Epoch 5/100
30/30 [==============================] - 0s 6ms/step - loss: 0.5741 - val_loss: 0.2904
Epoch 6/100
30/30 [==============================] - 0s 4ms/step - loss: 0.4654 - val_loss: 0.2515
Epoch 7/100
30/30 [==============================] - 0s 4ms/step - loss: 0.3727 - val_loss: 0.2247
Epoch 8/100
30/30 [==============================] - 0s 4ms/step - loss: 0.2782 - val_loss: 0.1842
Epoch 9/100
30/30 [==============================] - 0s 4ms/step - loss: 0.2180 - val_loss: 0.1551
Epoch 10/100
30/30 [==============================] - 0s 5ms/step - loss: 0.1740 - val_loss: 0.1320
Epoch 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
# Predict quality score with the new data
new_data = {
    "age": 59,
    "gender": 1,
    "stress_level": 10,
    "sleep_duration": 8.1,
    "bmi_underweight": 0,
    "bmi_normal": 0,
    "bmi_overweight": 1,
    "bmi_obese": 0
}

new_data_df = pd.DataFrame([new_data])
new_data_scaled = scaler.transform(new_data_df)
predicted_quality_score = nn_model.predict(new_data_scaled)
print(f"Predicted Quality Score: {predicted_quality_score[0][0]}")

1/1 [==============================] - 0s 19ms/step
Predicted Quality Score: 8.083221435546875
